<a href="https://colab.research.google.com/github/RockhoRockho/Scraping-Crawling/blob/main/CGV_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CGV 영화 리뷰 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # 눈에 띄게 보이지 않아도 띄우게끔
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def get_movie_reviews(url, page_num=10):

    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.get(url)

    writer_list = []
    review_list = []
    date_list =[]

    for page_no in range(1, page_num+1):
        try:
            page_ul = wd.find_element_by_id('paging_point')
            page_a = page_ul.find_element_by_link_text(str(page_no))
            page_a.click()
            time.sleep(1)

            writers = wd.find_elements_by_class_name('writer-name')
            writer_list += [writer.text for writer in writers]
            reviews = wd.find_elements_by_class_name('box-comment')
            review_list += [review.text for review in reviews]
            dates = wd.find_elements_by_class_name('day')
            date_list += [date.text for date in dates]

            if page_no % 10 == 0:
                next_button = page_ul.find_element_by_class_name('btn-paging next')
                next_button.click()
                time.sleep(1)
                
        except NoSuchElementException:
            break


    movie_review_df = pd.DataFrame({'Writer': writer_list,
                                    'Review': review_list,
                                    'Date': date_list})
    
    return movie_review_df

In [ ]:
url = 'http://www.cgv.co.kr/movies/detail-view/?midx=84690'
movie_review_df = get_movie_reviews(url, 12)
movie_review_df

## CGV 상영작 스크래핑

In [ ]:
url = 'http://www.cgv.co.kr/movies/'

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name('sect-movie-chart')
contents = movie_chart.find_elements_by_class_name('box-contents')
for content in contents:
    link = content.find_element_by_tag_name('a').get_attribute('href')
    title = content.find_element_by_class_name('title').text
    precent = content.find_element_by_class_name('percent').text
    info = content.find_element_by_class_name('txt-info').text
    print(title, precent, info, link)
    print(get_movie_reviews(link, 2))